In [1]:
import PyAstronomy as pyasl

In [ ]:
#check for observable transits
obsTime1 = Time('2019-02-17 1:00:00')
obsTime2 = Time('2019-02-22 14:00:00')
    
pdf = matplotlib.backends.backend_pdf.PdfPages("observableEvents.pdf")
good=np.zeros(len(info))
for i in range(len(info)):
    if info[i]['TESS Disposition']!='PC' or info[i]['TFOPWG Disposition'] in ['FP','CP']:
        continue

    ra=info[i]['RA (degrees)']
    dec=info[i]['Dec (degrees)']
    coordinates = SkyCoord(ra,dec,unit=(u.deg, u.deg))
    target = FixedTarget(coord=coordinates) 
    T0=Time(info[i]['Epoch (BJD)'],format='jd')
    P=info[i]['Period (days)']*u.day
    T14=info[i]['Duration (hours)']*u.hour
    if P==0:
        continue

    targetTr = EclipsingSystem(primary_eclipse_time=T0,orbital_period=P, duration=T14)
    midtransit_times = targetTr.next_primary_eclipse_time(obsTime1, n_eclipses=10)
    ingress_times,egress_times=targetTr.next_primary_ingress_egress_time(obsTime1,n_eclipses=10).T
    ingress_times,egress_times=ingress_times.flatten(),egress_times.flatten()
    
    constraints = [AtNightConstraint.twilight_astronomical(),AltitudeConstraint(min=30*u.deg),
                  TimeConstraint(min=obsTime1,max=obsTime2)]
    ingressOk=is_event_observable(constraints, observer, target, times=ingress_times).flatten()
    egressOk=is_event_observable(constraints, observer, target, times=egress_times).flatten()
    
    ok,=(ingressOk|egressOk).nonzero()
    for k in ok:
        print()
        print('TOI {}, TIC {}, comment={}'.format(info[i]['TOI'],info[i]['TIC ID'],info[i]['Comments']))
        print('depth={} mmag, T14={} h, star mag={},'.format(info[i]['Depth (mmag)'],T14.value,info[i]['TESS mag']))
        
        amIngress=observer.altaz(ingress_times[k],target).secz
        amEgress=observer.altaz(egress_times[k],target).secz
                
        if ingressOk[k]:
            eveTwilight=observer.twilight_evening_nautical(ingress_times[k],which='previous')
        else:
            eveTwilight=observer.twilight_evening_nautical(egress_times[k],which='previous')
        morTwilight=observer.twilight_morning_nautical(eveTwilight,which='next')

        trNo=np.floor((egress_times[k].jd-T0.jd)/P.value)
        T0_err=info[i]['Epoch error']
        P_err=info[i]['Period error']
        T14_err=info[i]['Duration error']
        midErr=np.sqrt(T0_err**2+trNo**2*P_err**2)
        ingressErr=np.sqrt((midErr*24)**2+0.25*T14_err**2)
        
        
        observe_time = eveTwilight-1.5*u.hour + (morTwilight-eveTwilight+3*u.hour)*np.linspace(0, 1, 75)
        fig=plt.figure(figsize=(8.5,11))
        ax=plt.axes([0.1,0.25,0.8,0.4])
        plot_airmass(target, observer, observe_time, altitude_yaxis=True, brightness_shading=True, ax=ax)
        plt.axvline(eveTwilight.plot_date,color='k')
        plt.axvline(morTwilight.plot_date,color='k')

        plt.axvspan((ingress_times[k]-ingressErr*u.hour).plot_date,
                    (ingress_times[k]+ingressErr*u.hour).plot_date,color='red',alpha=0.3,ec=None)
        plt.axvspan((egress_times[k]-ingressErr*u.hour).plot_date,
                    (egress_times[k]+ingressErr*u.hour).plot_date,color='blue',alpha=0.3,ec=None)

        plt.axvline(ingress_times[k].plot_date,color='red')
        plt.axvline(egress_times[k].plot_date,color='blue')
        
        #top axis in local time
        ax=plt.gca()
        ax2=plt.twiny()
        ax2.xaxis_date()
        ax2.set_xlim(ax.get_xlim())
        date_formatter = dates.DateFormatter('%H:%M',tz=tz)
        ax2.xaxis.set_major_formatter(date_formatter)
        _=plt.setp(ax2.get_xticklabels(), rotation=30, ha='left')
        _=ax2.set_xlabel('Local time')

#         plt.savefig(str(info[i]['TOI'])+'_'+str(k)+'.pdf')
#         plt.tight_layout()

        fig.text(0.05,0.92,'TOI {}, TIC {}'.format(info[i]['TOI'],info[i]['TIC ID']),fontsize=18)
        nKeys=len(keys)
        for j in range((nKeys+1)//2):
            key=keys[j]
            fig.text(0.05,0.88-j*0.02,'{}: {}'.format(key,info[i][key]))
        for j in range((nKeys+1)//2,nKeys):
            key=keys[j]
            fig.text(0.5,0.86-(j-(nKeys+1)//2)*0.02,'{}: {}'.format(key,info[i][key]))

        notes=[]
        notes.append(' observable? ingress={}, egress={}'.format(ingressOk[k],egressOk[k]))
        notes.append(' airmass at ingress: {}, at egress: {}'.format(amIngress,amEgress))
        notes.append(' ingress time: {}, egress time: {}'.format(ingress_times[k].iso,egress_times[k].iso))
        notes.append(' eve Twilight: {}, mor twilight: {}'.format(eveTwilight.iso,morTwilight.iso))
        notes.append(' transit time uncertainty: {} h'.format(midErr*24))
        notes.append(' transit ingress/error timing uncertainty: {} h'.format(ingressErr))
        for j,note in enumerate(notes):
            fig.text(0.05,0.16-j*0.02,note)
        
        pdf.savefig()
        plt.close()

        for note in notes:
            print(note)

    good[i]=ingressOk.any() or egressOk.any()

pdf.close()
good,=good.nonzero()